In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [2]:
from tqdm import tqdm
from pathlib import Path
from src.utils.config_loader import load_config

base_dir = Path(os.getcwd()).parent

config = load_config(base_dir / 'secrets.yaml')

In [3]:
from src.data.preprocessing import create_df

val_df = create_df(base_dir / 'data/my_data/regplans-dev.conllu')

In [4]:
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import (SystemMessage, HumanMessage)

os.environ['OPENAI_API_VERSION'] = config['OPENAI_API_VERSION']
os.environ['AZURE_OPENAI_ENDPOINT'] = config['OPENAI_API_BASE']
os.environ['AZURE_OPENAI_API_KEY'] = config['OPENAI_API_KEY']

llm = AzureChatOpenAI(
    deployment_name=config['OPENAI_DEPLOYMENT_NAME'],
    temperature=0.0
)

In [ ]:
from src.utils.label_mapping_regplans import label_to_id
from collections import defaultdict

all_pred_ids = []
all_true_ids = []
all_results = []

# Only do the anlaysis on 25 % of the data
val_df = val_df.iloc[:int(len(val_df) * 0.25)]

for idx, row in tqdm(val_df.iterrows(), total=len(val_df)):

    sentence = row['full_text']
    tokens = row['words']
    true_labels = row['labels']  

    msg = [
    SystemMessage(
        f"""You are an expert in Natural Language Processing. Your task is to identify Named Entities (NER) in a given text.
            The possible Named Entities are exclusively 'B-FELT' and 'I-FELT'. The entities are defined as follows:

            - B-FELT: The beginning of a field zone name.
            - I-FELT: The continuation of a field zone name.   
                        
            ### Examples:
            
            Example 1:
            Sentence: "Adkomst til BFS1 og BFS2 skal være fra Solfjellveien ."
            Entities:
            BFS1 B-FELT
            BFS2 B-FELT
                
            Example 2:
            Sentence: "På friområdene GF1 - GF3 tillates vanlig skjøtsel av trær og vegetasjon ."
            Entities:
            GF1 B-FELT
            - I-FELT
            GF3 I-FELT

            Example 3:
            Sentence: "Sonene med nemningane # 1 , # 2 og # 3 gjeld automatisk freda kulturminne , dyrkingsspor med id ."
            Entities:
            # B-FELT
            1 I-FELT
            # B-FELT
            2 I-FELT
            # B-FELT
            3 I-FELT

            Example 4:
            Sentence: "Areal brattere enn 1 : 3 , arealer i gul eller rød sone for henholdsvis støy ( T-1442 ) og luftkvalitet ( T-1520 ) ."
            Entities:

            ### Rules: 
            
            - Only tag words that belong to a field zone name.
            - If a field zone name consists of multiple tokens, the first token gets the 'B-FELT' label, and the rest get the 'I-FELT' label.
            - For a field zone name an I-FELT cannot exist without a preceding B-FELT.
            - Do not tag other words that are not field zone names.
            - Maintain the output format as follows: `{{entity}} {{label}}` (one per line).
        """
    ),
    HumanMessage(f"Your task is to identify the Named Entities in the following sentence: '{sentence}'") ]

    response = llm.invoke(msg)

    entities = defaultdict(list) # Word-label pairs

    for line in response.content.splitlines():
        parts = line.strip().split()
        if len(parts) == 2:
            word, label = parts[0], parts[1]
            entities[word].append(label)

    pred_labels = []
    word_counts = defaultdict(int)  # Track occurrences of each word

    for token in tokens:
        if token in entities and word_counts[token] < len(entities[token]):
            pred_labels.append(entities[token][word_counts[token]])  # Get the label in order
            word_counts[token] += 1  # Increment occurrence counter
        else:
            pred_labels.append("O")  # Default to "O" if missing

    # Convert labels to IDs
    pred_ids = []
    for label in pred_labels:
        if label in label_to_id:
            pred_ids.append(label_to_id[label])
        else:
            print(f"Warning: Unexpected label '{label}' found. Assigning default label 'O'.")
            pred_ids.append(label_to_id.get("O", -1))

    true_ids = [label_to_id[label] for label in true_labels]

    all_pred_ids.extend(pred_ids)
    all_true_ids.extend(true_ids)

    all_results.append({
        'sentence': sentence,
        'tokens': tokens,
        'true_labels': true_labels,
        'predicted_labels': pred_labels,
        'generated_text': response.content
    })

100%|██████████| 3/3 [00:01<00:00,  1.94it/s]


In [ ]:
from llm_stuff.evaluation import evaluate 
import json

metrics = evaluate(all_true_ids, all_pred_ids)

print("Evaluation Metrics on Dev Set:")
print(metrics)

final_output = {
    'prompt': str(msg),
    'evaluation_metrics': metrics,
    'results': all_results
}

with open(base_dir / f"llm_stuff/results/{config['OPENAI_DEPLOYMENT_NAME']}_PROMPT_V1.json", 'w', encoding='utf-8') as f:
    json.dump(final_output, f, indent=4, ensure_ascii=False)

Evaluation Metrics on Dev Set:
{'precision': 0.875, 'recall': 0.9878048896789551, 'f1': 0.922398567199707, 'span_acc': 1.0, 'classification_report': {'B-FELT': {'precision': 0.75, 'recall': 1.0, 'f1-score': 0.8571428571428571, 'support': 3.0}, 'O': {'precision': 1.0, 'recall': 0.975609756097561, 'f1-score': 0.9876543209876543, 'support': 41.0}, 'accuracy': 0.9772727272727273, 'macro avg': {'precision': 0.875, 'recall': 0.9878048780487805, 'f1-score': 0.9223985890652557, 'support': 44.0}, 'weighted avg': {'precision': 0.9829545454545454, 'recall': 0.9772727272727273, 'f1-score': 0.9787558120891453, 'support': 44.0}}}
